#Downloads

In [ ]:
!pip install --upgrade matplotlib

In [ ]:
!pip install --upgrade folium

#Imports

In [ ]:
from copy import copy
from datetime import datetime as dt
from google.colab import drive
from itertools import combinations
from matplotlib import cm
from scipy.stats import pearsonr as PEARSON_R
from sklearn.metrics import adjusted_rand_score as AR, fowlkes_mallows_score as FM, \
adjusted_mutual_info_score as AMI, normalized_mutual_info_score as NMI, v_measure_score as VM
from statsmodels.stats.weightstats import ztest as ztest

import folium
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sklearn.utils
import seaborn as sns

#Functions

In [ ]:
def read_and_arrange_df(path):
  df = pd.read_csv(path)
  df = df.drop('Unnamed: 0', 1)
  df['date'] = pd.to_datetime(df['date'])
  df.set_index('date', inplace=True)
  df.sort_index()

  return df

In [ ]:
def filter_df(index, years1, months1, years2, months2):
  df = dfs[index]['ma'][dfs[index]['ma'].index.year.isin(years1) & dfs[index]['ma'].index.month.isin(months1)]
  df.sort_index(inplace=True)
  if years2 != []:
    df_1 = dfs[index]['ma'][dfs[index]['ma'].index.year.isin(years2) & dfs[index]['ma'].index.month.isin(months2)]
    df_1.sort_index(inplace=True)
    df = pd.concat([df, df_1], axis=0)
    df.sort_index(inplace=True)
  df.dropna(inplace=True)

  _mean = dfs[index]['mean']
  _std = dfs[index]['std']

  return df, _mean, _std

In [ ]:
def find_transition_points(x_axis):
  points = []
  for i in range(len(x_axis) - 1):
    month_i = x_axis[i].split('-')[1]
    month_i_1 = x_axis[i + 1].split('-')[1]
    if (month_i == '04' and (month_i_1 == '05' or month_i_1 == '06' or month_i_1 == '11' or month_i_1 == '12')) \
    or ((month_i == '09' or month_i == '10') and (month_i_1 == '11' or month_i_1 == '12' or month_i_1 == '01' or month_i_1 == '04' or month_i_1 == '05' or month_i_1 == '06')): 
      points.append(x_axis[i + 1])
  return points

In [ ]:
def show_centroid_and_avg_graph(ts, df, _mean, _std):
  print(ts)
  color_winter = 'lavender'
  date_format = "%Y-%m-%d"
  cluster_col = dt.today().strftime(date_format)

  with open(dir + 'output/centroids/' + ts, 'rb') as f:
    cent = pickle.load(f)

  column_names = []
  for cluster in range(num_clusters):
    column_names.append('cluster_' + str(cluster))
  cent_df = pd.DataFrame(columns=column_names)

  for cluster in range(num_clusters):
    cent_df['cluster_' + str(cluster)] = (cent[cluster, :, 0].T * _std + _mean).tolist()

  cent_df.index = df.dropna().count(axis=1).index
  ##
  # df.index = df.dropna().count(axis=1).index

  x_axis = list(cent_df.index.format())
  ##
  # x_axis = list(df.index.format())

  plt.figure(figsize=(54, 9))
  for cluster in range(num_clusters):
    plt.plot(x_axis, 
             cent_df['cluster_' + str(cluster)].tolist(), 
             label='cluster_' + str(cluster) + '_centroid', 
             color=cluster_color[cluster], 
             linewidth=1,
             linestyle='dashed',
             marker='|')
  
  df_t = df.T
  df_t[cluster_col] = points_df[ts + '__' + str(num_clusters)].values
  df_t_mean = df_t.groupby(cluster_col).mean()
  # df_t_mean = df_t.groupby(cluster_col).median()

  for cluster in range(num_clusters):
    plt.plot(x_axis, 
             df_t_mean.iloc[cluster, :].tolist(), 
             label='cluster_' + str(cluster) + '_avg', 
             color=cluster_color[cluster], 
             linewidth=3,
             linestyle='solid',    
             marker='o')
  
  plt.xticks(rotation=67.5)
  plt.legend()
  
  plt.xlabel('Time')
  y_label = ts_clusters.split('__')[1]
  if y_label == 'precipitation':
    y_label += ' (weekly, total) [mm]'
  if y_label == 'lst_c':
    y_label = 'lst (weekly, average) [C]'
  if y_label == 'lst_k':
    y_label = 'lst (weekly, average) [K]'
  plt.ylabel(y_label)

  transition_points = find_transition_points(x_axis)
  if transition_points and transition_points[-1].split('-')[0] != '2021':
    transition_points.append('2021-11-01')
  print(transition_points)
  for i in range(len(transition_points) - 1):
    if i % 2 == 0:
      if dt.strptime(transition_points[i + 1], date_format) < dt.strptime(x_axis[-1], date_format):
        plt.axvspan(transition_points[i], transition_points[i + 1], color=color_winter)
      else:
        plt.axvspan(transition_points[i], x_axis[-1], color=color_winter)
  
  plt.title(f"Clusters' Centroids' and Avg. Time-Series: {ts}\n (# Samples = {len(x_axis)})")
  plt.grid(color='lightgray', linewidth=0.5)
  plt.savefig(dir + 'output/centroids_avg_graphs/' + ts + '.jpg', transparent=True, bbox_inches='tight')
  plt.show()
  print()

In [ ]:
def show_cluster_counts(ts_clusters):
  ax = sns.countplot(x=ts_clusters, data=points_df, palette=list(cluster_color.values()))
  ax.set_title("Clusters: " + '__'.join(ts_clusters.split('__')[:-1]))
  ax.set_xlabel('Cluster')
  ax.bar_label(container=ax.containers[0])
  plt.savefig(dir + 'output/cluster_counts/' + '__'.join(ts_clusters.split('__')[:-1]) + '.jpg', transparent=True, bbox_inches='tight')
  plt.show()
  print()

In [ ]:
def draw_square():
  W = points_df['lon'].min() - 0.1
  E = points_df['lon'].max() + 0.1
  S = points_df['lat'].min() - 0.1
  N = points_df['lat'].max() + 0.1
  
  upper_left = (N, W)
  upper_right = (N, E)
  lower_right = (S, E)
  lower_left = (S, W)
  line_color='gainsboro'
  weight=2
  text='text'
  edges = [upper_left, upper_right, lower_right, lower_left]

  return edges, line_color, weight

In [ ]:
def circle_cluster_map(ts_clusters):
  map = folium.Map(location=[points_df.lat.mean(), points_df.lon.mean()], 
                   zoom_start=11, 
                   control_scale=True,
                   tiles="Stamen Toner")
  points_df.apply(lambda row: folium.CircleMarker(location=[row["lat"], row["lon"]], 
                                                  # radius=5,
                                                  radius=1,
                                                  weight=0,
                                                  fill=True,
                                                  # color=cluster_color[row[ts_clusters]],
                                                  fill_color=cluster_color[row[ts_clusters]],
                                                  # opacity=0.25,
                                                  # fill_opacity=0.25
                                                  fill_opacity=1).add_to(map), 
                  axis=1)
  
  edges, line_color, weight = draw_square()
  map.add_child(folium.vector_layers.Polygon(locations=edges, color=line_color, weight=weight))

  return map

In [ ]:
def show_heatmap(df, reg, metric, annot_size, thresh, title):
  df_hm = df.filter(regex=reg)
  df_hm = df_hm.reindex(sorted(df_hm.columns), axis=1)

  arr = np.zeros((df_hm.shape[1], df_hm.shape[1]))
  if metric is not PEARSON_R:
    for i in range(arr.shape[0]):
      for j in range(arr.shape[1]):
        arr[i, j] = metric(df_hm.iloc[:, i], df_hm.iloc[:, j])

    my_cmap = copy(plt.cm.coolwarm)
    v_thresh = thresh
    if thresh == -2:
      v_thresh = np.amin(arr)
  
  else:
    for i in range(arr.shape[0]):
      for j in range(arr.shape[1]):
        r, p = metric(df_hm.iloc[:, i], df_hm.iloc[:, j])
        if p < 0.05:
          arr[i, j] = r
        else:
          arr[i, j] = -1.00001
    
    my_cmap = copy(plt.cm.bwr)
    v_thresh = -1.000001

  my_cmap.set_under("black")

  strings = np.empty([df_hm.shape[1], df_hm.shape[1]], dtype=object)
  for i in range(strings.shape[0]):
    for j in range(strings.shape[1]):
      # Bootstrap
      metric_list = []  
      l1 = df_hm.iloc[:, i]
      l2 = df_hm.iloc[:, j]
      for k in range(1000):
        ll1 = sklearn.utils.shuffle(l1)
        ll2 = sklearn.utils.shuffle(l2)
        metric_list.append(metric(ll1, ll2))

      if ztest(metric_list, value=arr[i, j])[1] <= 0.01:
        strings[i, j] = "**"
      else:
        if ztest(metric_list, value=arr[i, j])[1] <= 0.05:
          strings[i, j] = "*"
        else:
          strings[i, j] = ""

  labels = (np.asarray(["{0}\n{1:.3f}".format(string, value)
  for string, value in zip(strings.flatten(),
                           arr.flatten())])
  ).reshape(df_hm.shape[1], df_hm.shape[1])

  fig, ax = plt.subplots(figsize=(20, 16))
  # ax = sns.heatmap(arr, annot=True, cmap=my_cmap, annot_kws={"color": "black", "size": annot_size}, vmin=v_thresh)
  ax = sns.heatmap(arr, annot=labels, fmt="", cmap=my_cmap, annot_kws={"color": "black", "size": annot_size}, vmin=v_thresh)

  ax.set_xticklabels(df_hm.columns, rotation=90)
  ax.set_yticklabels(df_hm.columns, rotation=0)
  ax.set_title("Heatmap: " + title)
  if metric is FM:
    plt.savefig(dir + 'output/FM/' + reg + '.jpg', transparent=True, bbox_inches='tight')
  plt.show()
  print()

In [ ]:
def scatter_plot_with_stats(df, x, y, thresh, cluster_x, cluster_y):
  df = df.dropna(subset=[x, y])
  N = df.shape[0]
  r, p = PEARSON_R(df[x], df[y])
  if abs(r) < thresh:
    if cluster_x == -1 and cluster_y == -1:
      # print(f'Correlation (Pearson) of {x} and {y} is less than {thresh}. Exiting...')
      return
    else:
      # print(f'Correlation (Pearson) of {x} (cluster={cluster_x}) and {y} (cluster={cluster_y}) is less than {thresh}. Exiting...')
      return
  else:
    if p > 0.05:
      # print(f'P-value of {x} and {y} is greater than 0.05. Exiting...')
      return
    else:
      corr_bins = {0: 'red', 0.3: 'orange', 0.5: 'yellow', 0.7: 'green', 0.9: 'blue'}
      col = corr_bins.get(abs(r)) or corr_bins[min(corr_bins.keys(), key = lambda key: abs(key - abs(r)))]
      graph = sns.jointplot(data=df, x=x, y=y, kind='reg', joint_kws={'line_kws':{'color':col}})
      if cluster_x == -1 and cluster_y == -1:
        plt.suptitle(f'{x} vs. {y}\n(# Points = {N})', y=1.05)
      else:
        plt.suptitle(f'{x} (cluster={cluster_x}) vs. {y} (cluster={cluster_y})\n(# Points = {N})', y=1.05)
      phantom, = graph.ax_joint.plot([], [], linestyle="", alpha=0)
      graph.ax_joint.legend([phantom], ['r={:f}, p={:e}'.format(r, p)])
      plt.show()
      print()

In [ ]:
def create_index_bins(df):
  avg_val = np.nanmean(df)
  std_val = np.nanstd(df)
  min_val = max(avg_val - 3 * std_val, np.nanmin(df))
  max_val = min(avg_val + 3 * std_val, np.nanmax(df))

  step = (max_val - min_val) / (num_bins - 1)

  index_bins = list(np.arange(min_val, max_val + step, step))
  return index_bins

In [ ]:
def get_closest_index(val, bins):
  abs_diff_func = lambda bin_val : abs(bin_val - val)
  closest_val = min(bins, key=abs_diff_func)
  return int(bins.index(closest_val))

In [ ]:
def get_hex_color(i, cluster):
  rgba = cmaps[cluster](i)
  return matplotlib.colors.rgb2hex(rgba)

In [ ]:
def index_statistic_map(df, ts, index_bins, statistic):
  map = folium.Map(location=[points_df.lat.mean(), points_df.lon.mean()], 
                   zoom_start=11, 
                   control_scale=True,
                   tiles="Stamen Toner")

  for i in range(num_points):
    folium.CircleMarker(location=[points_df['lat'][i], points_df['lon'][i]],
                        # radius=4,
                        radius=1,
                        weight=0,
                        fill=True,
                        fill_color=get_hex_color(get_closest_index(df[ts + '__' + statistic][i], index_bins), df[ts + '__cluster'][i]),
                        # fill_opacity=0.5
                        fill_opacity=1).add_to(map)

  edges, line_color, weight = draw_square()
  map.add_child(folium.vector_layers.Polygon(locations=edges, color=line_color, weight=weight))

  return map

#Mount Drive

In [ ]:
drive.mount('/content/drive')

#Constants

In [ ]:
dir = "/content/drive/MyDrive/Thesis/"

In [ ]:
cluster_color = {0: 'red', 1: 'lime', 2: 'blue', 3: 'orange', 4: 'pink', 5: 'purple'}

In [ ]:
metrics = [FM]
# metrics = [AR, FM, AMI, NMI, VM, PEARSON_R]

In [ ]:
corr_thresh = 0.5  # 0.5-0.69: moderate, 0.7-0.89: high, 0.9: very high

In [ ]:
years = [2017, 2018, 2019, 2020, 2021]
months_year_1 = [5, 6, 7, 8, 9, 10, 11, 12]
months_year_2 = [1, 2, 3, 4]
months_summer = [5, 6, 7, 8, 9, 10]
months_winter_1 = [11, 12]
months_winter_2 = [1, 2, 3, 4]

In [ ]:
num_points = 20000

In [ ]:
num_bins = 13

In [ ]:
cmaps = [cm.get_cmap('Reds', num_bins),
         cm.get_cmap('Greens', num_bins),
         cm.get_cmap('Blues', num_bins),
         cm.get_cmap('Oranges', num_bins),
         cm.get_cmap('pink', num_bins),
         cm.get_cmap('Purples', num_bins)]

#Code

In [ ]:
points_df = pd.read_csv(dir + 'output/points.csv')
points_df = points_df.drop('Unnamed: 0', 1)

df_temp = points_df[['lat', 'lon']]
points_df.drop(['lat', 'lon'], axis=1, inplace=True)
points_df = points_df.reindex(sorted(points_df.columns), axis=1)
points_df = pd.concat([df_temp, points_df], axis=1)

print(points_df)

In [ ]:
print(points_df['lat'].mean())
print(points_df['lon'].mean())

In [ ]:
points_df.columns

In [ ]:
clusters = sorted(list(points_df.iloc[:, 2].unique()))
clusters

In [ ]:
num_clusters = len(clusters)
num_clusters

In [ ]:
indices_and_sources = {'precipitation': 'CHIRPS', 'lst_c': 'MODIS', 'b_asc_vh': 'S1', 
                       'ci': 'S2', 'ndvi': 'S2', 'ndwi': 'S2'}
dfs = {}

for index, source in indices_and_sources.items():
  print(source + '__' + index)
  print("=====================")
  dfs[source + '__' + index] = {}
  dfs[source + '__' + index]['ma'] = read_and_arrange_df(dir + source + '/' + index + '__ma.csv')
  dfs[source + '__' + index]['mean'] = np.nanmean(dfs[source + '__' + index]['ma'].values)
  print(f"mean = {dfs[source + '__' + index]['mean']}")
  dfs[source + '__' + index]['std'] = np.nanstd(dfs[source + '__' + index]['ma'].values)
  print(f"std = {dfs[source + '__' + index]['std']}\n")

In [ ]:
indices = list(dfs.keys())
indices

In [ ]:
print(dfs['S1__b_asc_vh']['ma'].head(30))

In [ ]:
# !cp /content/drive/MyDrive/Thesis/output/FM/summer_each_year.jpg /content/drive/MyDrive/Thesis/output_copy/FM/summer_each_year.jpg

##Clusterings' Results' Analysis (Maps, Counts, Metrics)

###1.&nbsp;4.5 Years (May 2017 - Oct 2021)

In [ ]:
gen_reg = "2017_05__2021_10"
ts_clusters_list = list(points_df.filter(regex=gen_reg).columns)
ts_clusters_list

In [ ]:
len(ts_clusters_list)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  index = '__'.join(ts_clusters.split('__')[:2])
  df, _mean, _std = filter_df(index, years, months_year_2 + months_year_1, [], [])
  ts = '__'.join(ts_clusters.split('__')[:-1])
  show_centroid_and_avg_graph(ts, df, _mean, _std)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  show_cluster_counts(ts_clusters)
  # circle_cluster_map(ts_clusters)  # Need to do manually

In [ ]:
ts_clusters_list

In [ ]:
i = 3
print(ts_clusters_list[i])
circle_cluster_map(ts_clusters_list[i])  # Need to do manually

In [ ]:
# thresh = 0.475
# thresh = 0.54
thresh = -2

for metric in metrics:
  show_heatmap(points_df, gen_reg, metric, 15, thresh, "** " + metric.__name__ + " (Clusters) **" + "\n4.5 Years (May 2017 - Oct 2021)")

In [ ]:
df = points_df[['lat', 'lon']]
for index in indices:
  df_temp = dfs[index]['ma']
  statistics = {'max': df_temp.max(0, True).values, 'mean': df_temp.mean(0, True).values, 'std': df_temp.std(0, True).values, 'var': df_temp.var(0, True).values}
  reg_ix = index + "__" + gen_reg  # Need to change to 'reg' later
  df[reg_ix + '__cluster'] = points_df.filter(regex=reg_ix).iloc[:, 0].values
  for statistic in statistics:
    df[reg_ix + '__' + statistic] = statistics[statistic]

df_temp = df[['lat', 'lon']]
df.drop(['lat', 'lon'], axis=1, inplace=True)
df = df.reindex(sorted(df.columns), axis=1)
df = pd.concat([df_temp, df], axis=1)

In [ ]:
df

In [ ]:
statistics.keys()

In [ ]:
for st in statistics.keys():
  show_heatmap(df, gen_reg + '__' + st, PEARSON_R, 15, thresh, f"** {PEARSON_R.__name__} ({st}) **\n4.5 Years (May 2017 - Oct 2021)")

In [ ]:
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  pairs = list(combinations(s_list, 2))
  # print(pairs)
  # print(len(pairs))
  for i, pair in enumerate(pairs, start=1):
    # print(i)
    df1 = df
    scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, -1, -1)
    for c_x in clusters:
      for c_y in clusters:
        df1 = df[(df[pair[0].replace(st, 'cluster')] == c_x) & (df[pair[1].replace(st, 'cluster')] == c_y)]
        try:
          scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, c_x, c_y)
        except:
          # print(f'Not enough information for: {pair[0]} cluster = {c_x}, and: {pair[1]} cluster = {c_y}.')
          pass

In [ ]:
i = 0
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  # print(s_list)
  for s in s_list:
    print(i)

    ax = sns.histplot(data=df, x=s)
    ax.set_title("Histogram: " + s + " (all points)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={s.replace(st, 'cluster'): 'Cluster'}, inplace=True)

    ax = sns.histplot(data=df, x=s, hue='Cluster', palette=list(cluster_color.values())[:num_clusters])
    ax.set_title("Histogram: " + s + " (by clusters)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={'Cluster': s.replace(st, 'cluster')}, inplace=True)
    i += 1

In [ ]:
ts_list = []

for index in indices:
  # print(index)
  # print('-------------')
  for st in statistics.keys():
    # print(st)
    # print('----')
    ts = index + "__" + gen_reg + "__" + st  # Need to change to 'reg' later
    ts_list.append(ts)
    # # Need to do manually
    # df1 = df.filter(regex=ts)
    # index_bins = create_index_bins(df1)
    # avg_index_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
ts_list

In [ ]:
len(ts_list)

In [ ]:
i = 13

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
i = 4

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

###2.&nbsp;Seasons (May-Oct, Nov-Apr; 2 Periods)

In [ ]:
gen_reg = "__05_10|11_04"
ts_clusters_list = list(points_df.filter(regex=gen_reg).columns)
ts_clusters_list

In [ ]:
len(ts_clusters_list)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  index = '__'.join(ts_clusters.split('__')[:2])

  months = ts_clusters.split('__')[2]
  if months == '05_10':  # Summers
    df, _mean, _std = filter_df(index, years, months_summer, [], [])
  else:  # Winters ('11_04')
    df, _mean, _std = filter_df(index, years, months_winter_1, years, months_winter_2)
  
  ts = '__'.join(ts_clusters.split('__')[:-1])
  show_centroid_and_avg_graph(ts, df, _mean, _std)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  show_cluster_counts(ts_clusters)
  # circle_cluster_map(ts_clusters)  # Need to do manually

In [ ]:
ts_clusters_list

In [ ]:
i = 11
print(ts_clusters_list[i])
circle_cluster_map(ts_clusters_list[i])  # Need to do manually

In [ ]:
dir

In [ ]:
# thresh = 0.54
thresh = -2

for metric in metrics:
  show_heatmap(points_df, gen_reg, metric, 10, thresh, "** " + metric.__name__ + " (Clusters) **" + "\nSeasons (May-Oct, Nov-Apr 2017-2021)")

In [ ]:
df = points_df[['lat', 'lon']]
for index in indices:
  df_temp, _, _ = filter_df(index, years, months_summer, [], [])
  statistics = {'max': df_temp.max(0, True).values, 'mean': df_temp.mean(0, True).values, 'std': df_temp.std(0, True).values, 'var': df_temp.var(0, True).values}
  reg = f'05_10'
  reg_ix = index + "__" + reg
  df[reg_ix + '__cluster'] = points_df.filter(regex=reg_ix).iloc[:, 0].values
  for statistic in statistics:
    df[reg_ix + '__' + statistic] = statistics[statistic]

  df_temp, _, _ = filter_df(index, years, months_winter_1, years, months_winter_2)
  statistics = {'max': df_temp.max(0, True).values, 'mean': df_temp.mean(0, True).values, 'std': df_temp.std(0, True).values, 'var': df_temp.var(0, True).values}
  reg = f'11_04'
  reg_ix = index + "__" + reg
  df[reg_ix + '__cluster'] = points_df.filter(regex=reg_ix).iloc[:, 0].values
  for statistic in statistics:
    df[reg_ix + '__' + statistic] = statistics[statistic]

df_temp = df[['lat', 'lon']]
df.drop(['lat', 'lon'], axis=1, inplace=True)
df = df.reindex(sorted(df.columns), axis=1)
df = pd.concat([df_temp, df], axis=1)

In [ ]:
df

In [ ]:
statistics.keys()

In [ ]:
for st in statistics.keys():
  r = gen_reg.split('|')[0] + '__' + st + '|' + gen_reg.split('|')[1] + '__' + st
  show_heatmap(df, r, PEARSON_R, 10, thresh, f"** {PEARSON_R.__name__} ({st}) **\nSeasons (May-Oct, Nov-Apr 2017-2021)")

In [ ]:
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  pairs = list(combinations(s_list, 2))
  # print(pairs)
  # print(len(pairs))
  for i, pair in enumerate(pairs, start=1):
    # print(i)
    df1 = df
    scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, -1, -1)
    for c_x in clusters:
      for c_y in clusters:
        df1 = df[(df[pair[0].replace(st, 'cluster')] == c_x) & (df[pair[1].replace(st, 'cluster')] == c_y)]
        try:
          scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, c_x, c_y)
        except:
          # print(f'Not enough information for: {pair[0]} cluster = {c_x}, and: {pair[1]} cluster = {c_y}.')
          pass

In [ ]:
i = 0
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  # print(s_list)
  for s in s_list:
    print(i)

    ax = sns.histplot(data=df, x=s)
    ax.set_title("Histogram: " + s + " (all points)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={s.replace(st, 'cluster'): 'Cluster'}, inplace=True)

    ax = sns.histplot(data=df, x=s, hue='Cluster', palette=list(cluster_color.values())[:num_clusters])
    ax.set_title("Histogram: " + s + " (by clusters)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={'Cluster': s.replace(st, 'cluster')}, inplace=True)
    i += 1

In [ ]:
ts_list = []

for index in indices:
  # print(index)
  # print('-------------')
  for st in statistics.keys():
    # print(st)
    # print('----')
    reg = f'05_10'
    ts = index + "__" + reg + "__" + st
    ts_list.append(ts)
    
    reg = f'11_04'
    ts = index + "__" + reg + "__" + st
    ts_list.append(ts)
    # # Need to do manually
    # df1 = df.filter(regex=ts)
    # index_bins = create_index_bins(df1)
    # avg_index_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
ts_list

In [ ]:
len(ts_list)

In [ ]:
i = 18

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + gen_reg.split('|')[0] + '__' + st + '|' + '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[1] + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
i = 19

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + gen_reg.split('|')[0] + '__' + st + '|' + '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[1] + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

###3.&nbsp;Years (May-Apr; 4 Years)

In [ ]:
# points_df = points_df.filter(regex='CHIRPS|ci|ndvi|ndwi')
# points_df

In [ ]:
gen_reg = "20\d\d_05__20\d\d_04"
ts_clusters_list = list(points_df.filter(regex=gen_reg).columns)
ts_clusters_list

In [ ]:
len(ts_clusters_list)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  index = '__'.join(ts_clusters.split('__')[:2])

  year1 = int(ts_clusters.split('__')[2].split('_')[0])
  year2 = int(ts_clusters.split('__')[3].split('_')[0])
  df, _mean, _std = filter_df(index, [year1], months_year_1, [year2], months_year_2)
  ts = '__'.join(ts_clusters.split('__')[:-1])
  show_centroid_and_avg_graph(ts, df, _mean, _std)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  show_cluster_counts(ts_clusters)
  # circle_cluster_map(ts_clusters)  # Need to do manually

In [ ]:
ts_clusters_list

In [ ]:
i = 23
print(ts_clusters_list[i])
circle_cluster_map(ts_clusters_list[i])  # Need to do manually

In [ ]:
# thresh = 0.515
thresh = -2

for metric in metrics:
  show_heatmap(points_df, gen_reg, metric, 10, thresh, "** " + metric.__name__ + " (Clusters) **" + "\nYears (May-Apr 2017-2021)")

In [ ]:
df = points_df[['lat', 'lon']]
for index in indices:
  for i in range(len(years) - 1):
    df_temp, _, _ = filter_df(index, [years[i]], months_year_1, [years[i + 1]], months_year_2)
    statistics = {'max': df_temp.max(0, True).values, 'mean': df_temp.mean(0, True).values, 'std': df_temp.std(0, True).values, 'var': df_temp.var(0, True).values}
    reg = f'{years[i]}_05__{years[i + 1]}_04'
    reg_ix = index + "__" + reg
    df[reg_ix + '__cluster'] = points_df.filter(regex=reg_ix).iloc[:, 0].values
    for statistic in statistics:
      df[reg_ix + '__' + statistic] = statistics[statistic]

df_temp = df[['lat', 'lon']]
df.drop(['lat', 'lon'], axis=1, inplace=True)
df = df.reindex(sorted(df.columns), axis=1)
df = pd.concat([df_temp, df], axis=1)

In [ ]:
df

In [ ]:
statistics.keys()

In [ ]:
for st in statistics.keys():
  show_heatmap(df, gen_reg + '__' + st, PEARSON_R, 10, thresh, f"** {PEARSON_R.__name__} ({st}) **\nYears (May-Apr 2017-2021)")

In [ ]:
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  pairs = list(combinations(s_list, 2))
  # print(pairs)
  # print(len(pairs))
  for i, pair in enumerate(pairs, start=1):
    # print(i)
    df1 = df
    scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, -1, -1)
    for c_x in clusters:
      for c_y in clusters:
        df1 = df[(df[pair[0].replace(st, 'cluster')] == c_x) & (df[pair[1].replace(st, 'cluster')] == c_y)]
        try:
          scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, c_x, c_y)
        except:
          # print(f'Not enough information for: {pair[0]} cluster = {c_x}, and: {pair[1]} cluster = {c_y}.')
          pass

In [ ]:
i = 0
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  # print(s_list)
  for s in s_list:
    print(i)

    ax = sns.histplot(data=df, x=s)
    ax.set_title("Histogram: " + s + " (all points)")
    ax.set(xlabel='Value')
    plt.show()
    print()
    
    df.rename(columns={s.replace(st, 'cluster'): 'Cluster'}, inplace=True)

    ax = sns.histplot(data=df, x=s, hue='Cluster', palette=list(cluster_color.values())[:num_clusters])
    ax.set_title("Histogram: " + s + " (by clusters)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={'Cluster': s.replace(st, 'cluster')}, inplace=True)
    i += 1

In [ ]:
ts_list = []

for index in indices:
  # print(index)
  # print('-------------')
  for st in statistics.keys():
    # print(st)
    # print('----')
    for i in range(len(years) - 1):
      reg = f'{years[i]}_05__{years[i + 1]}_04'
      ts = index + "__" + reg + "__" + st
      ts_list.append(ts)
    # # Need to do manually
    # df1 = df.filter(regex=ts)
    # index_bins = create_index_bins(df1)
    # avg_index_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
ts_list

In [ ]:
len(ts_list)

In [ ]:
i = 38

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
i = 39

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

###4.&nbsp;Seasons over the Years (May-Oct, Nov-Apr 2017-2021; 9 Periods)

In [ ]:
# points_df = points_df.filter(regex='CHIRPS|ci|ndvi')
# points_df

In [ ]:
gen_reg = "20\d\d_05_10|20\d\d_11__20\d\d_04"
ts_clusters_list = list(points_df.filter(regex=gen_reg).columns)
ts_clusters_list

In [ ]:
len(ts_clusters_list)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  index = '__'.join(ts_clusters.split('__')[:2])

  month1 = ts_clusters.split('__')[2].split('_')[1]
  year1 = int(ts_clusters.split('__')[2].split('_')[0])
  if month1 == '05':  # Summer (May-Oct)
    df, _mean, _std = filter_df(index, [year1], months_summer, [], [])
  else:  # Winter (Nov-Apr)
    df, _mean, _std = filter_df(index, [year1], months_winter_1, [year1 + 1], months_winter_2)
  ts = '__'.join(ts_clusters.split('__')[:-1])
  show_centroid_and_avg_graph(ts, df, _mean, _std)

In [ ]:
for i, ts_clusters in enumerate(ts_clusters_list):
  print(i)
  show_cluster_counts(ts_clusters)
  # circle_cluster_map(ts_clusters)  # Need to do manually

In [ ]:
ts_clusters_list

In [ ]:
i = 53
print(ts_clusters_list[i])
circle_cluster_map(ts_clusters_list[i])  # Need to do manually

In [ ]:
i = 25
print(ts_clusters_list[i])
circle_cluster_map(ts_clusters_list[i])  # Need to do manually

In [ ]:
i = 26
print(ts_clusters_list[i])
circle_cluster_map(ts_clusters_list[i])  # Need to do manually

In [ ]:
thresh = -2

for metric in metrics:
  # show_heatmap(points_df, gen_reg, metric, 5, "** " + metric.__name__ + " (Clusters) **" + "\nSeasons over the Years (May-Oct, Nov-Apr 2017-2021)")
  show_heatmap(points_df, gen_reg, metric, 5, thresh, "** " + metric.__name__ + " (Clusters) **" + "\nSeasons over the Years (May-Oct, Nov-Apr 2017-2021)")

In [ ]:
thresh = -2

for metric in metrics:
  show_heatmap(points_df, gen_reg.split('|')[0], metric, 7, thresh, "** " + metric.__name__ + " (Clusters) **" + "\nSummers over the Years (May-Oct 2017-2021)")

In [ ]:
thresh = -2

for metric in metrics:
  show_heatmap(points_df, gen_reg.split('|')[1], metric, 7, thresh, "** " + metric.__name__ + " (Clusters) **" + "\nWinters over the Years (Nov-Apr 2017-2021)")

In [ ]:
df = points_df[['lat', 'lon']]
for index in indices:
  for i in range(len(years)):
    df_temp, _, _ = filter_df(index, [years[i]], months_summer, [], [])
    statistics = {'max': df_temp.max(0, True).values, 'mean': df_temp.mean(0, True).values, 'std': df_temp.std(0, True).values, 'var': df_temp.var(0, True).values}
    reg = f'{years[i]}_05_10'
    reg_ix = index + "__" + reg
    df[reg_ix + '__cluster'] = points_df.filter(regex=reg_ix).iloc[:, 0].values
    for statistic in statistics:
      df[reg_ix + '__' + statistic] = statistics[statistic]

  for i in range(len(years) - 1):
    df_temp, _, _ = filter_df(index, [years[i]], months_winter_1, [years[i + 1]], months_winter_2)
    statistics = {'max': df_temp.max(0, True).values, 'mean': df_temp.mean(0, True).values, 'std': df_temp.std(0, True).values, 'var': df_temp.var(0, True).values}
    reg = f'{years[i]}_11__{years[i + 1]}_04'
    reg_ix = index + "__" + reg
    df[reg_ix + '__cluster'] = points_df.filter(regex=reg_ix).iloc[:, 0].values
    for statistic in statistics:
      df[reg_ix + '__' + statistic] = statistics[statistic]

df_temp = df[['lat', 'lon']]
df.drop(['lat', 'lon'], axis=1, inplace=True)
df = df.reindex(sorted(df.columns), axis=1)
df = pd.concat([df_temp, df], axis=1)

In [ ]:
df

In [ ]:
statistics.keys()

In [ ]:
for st in statistics.keys():
  r = gen_reg.split('|')[0] + '__' + st + '|' + gen_reg.split('|')[1] + '__' + st
  show_heatmap(df, r, PEARSON_R, 5, thresh, f"** {PEARSON_R.__name__} ({st}) **\nSeasons over the Years (May-Oct, Nov-Apr 2017-2021)")

In [ ]:
for st in statistics.keys():
  r = gen_reg.split('|')[0] + '__' + st
  show_heatmap(df, r, PEARSON_R, 7, thresh, f"** {PEARSON_R.__name__} ({st}) **\nSummers over the Years (Nov-Apr 2017-2021)")

In [ ]:
for st in statistics.keys():
  r = gen_reg.split('|')[1] + '__' + st
  show_heatmap(df, r, PEARSON_R, 7, thresh, f"** {PEARSON_R.__name__} ({st}) **\nWinters over the Years (Nov-Apr 2017-2021)")

In [ ]:
# for st in statistics.keys():
#   print(st)
#   print('----')
#   s_list = list(df.filter(regex=st+'$').columns)
#   pairs = list(combinations(s_list, 2))
#   # print(pairs)
#   # print(len(pairs))
#   for i, pair in enumerate(pairs, start=1):
#     # print(i)
#     df1 = df
#     scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, -1, -1)
#     for c_x in clusters:
#       for c_y in clusters:
#         df1 = df[(df[pair[0].replace(st, 'cluster')] == c_x) & (df[pair[1].replace(st, 'cluster')] == c_y)]
#         try:
#           scatter_plot_with_stats(df1, pair[0], pair[1], corr_thresh, c_x, c_y)
#         except:
#           # print(f'Not enough information for: {pair[0]} cluster = {c_x}, and: {pair[1]} cluster = {c_y}.')
#           pass

In [ ]:
i = 0
for st in statistics.keys():
  print(st)
  print('----')
  s_list = list(df.filter(regex=st+'$').columns)
  print(s_list)
  for s in s_list:
    print(i)

    ax = sns.histplot(data=df, x=s)
    ax.set_title("Histogram: " + s + " (all points)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={s.replace(st, 'cluster'): 'Cluster'}, inplace=True)

    ax = sns.histplot(data=df, x=s, hue='Cluster', palette=list(cluster_color.values())[:num_clusters])
    ax.set_title("Histogram: " + s + " (by clusters)")
    ax.set(xlabel='Value')
    plt.show()
    print()

    df.rename(columns={'Cluster': s.replace(st, 'cluster')}, inplace=True)
    i += 1

In [ ]:
ts_list = []

for index in indices:
  # print(index)
  # print('-------------')
  for st in statistics.keys():
    # print(st)
    # print('----')
    for i in range(len(years)):
      reg = f'{years[i]}_05_10'
      ts = index + "__" + reg + "__" + st
      ts_list.append(ts)
    
    for i in range(len(years) - 1):
      reg = f'{years[i]}_11__{years[i + 1]}_04'
      ts = index + "__" + reg + "__" + st
      ts_list.append(ts)
    # # Need to do manually
    # df1 = df.filter(regex=ts)
    # index_bins = create_index_bins(df1)
    # avg_index_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
ts_list

In [ ]:
len(ts_list)

In [ ]:
i = 84

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[0] + '__' + st + '|' + '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[1] + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
i = 89

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[0] + '__' + st + '|' + '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[1] + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

In [ ]:
i = 85

ts = '__'.join(ts_list[i].split('__')[:-1])
print(ts)
st = ts_list[i].split('__')[-1]
print(st)
cols = '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[0] + '__' + st + '|' + '__'.join(ts_list[i].split('__')[:2]) + '__' + gen_reg.split('|')[1] + '__' + st
print(cols)
df1 = df.filter(regex=cols)
print(df1.columns)
index_bins = create_index_bins(df1)
print(index_bins)
index_statistic_map(df, ts, index_bins, st)  # Need to do manually

###End